## Import Libraries

In [1]:
import tensorflow_hub as hub
import pandas as pd
import tensorflow_text as text
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np

## Read Data

In [2]:
# load data
df = pd.read_csv('spam.csv', encoding = "ISO-8859-1")
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


## Exploratory Data Analysis


In [3]:
# check count and unique and top values and their frequency
df['v1'].value_counts()

ham     4825
spam     747
Name: v1, dtype: int64

## Downsampling Data

Downsampling is a technique where the majority class is downsampled to match the minority class. Since our data has only one column(feature) it ok to use it.

We perform downsampling by just picking any random 747 samples from the ham class

In [4]:
# check percentage of data - states how much data needs to be balanced
print(str(round(747/4825,2))+'%')

0.15%


## filtering and separation

In [5]:
# creating 2 new dataframe as df_ham , df_spam

df_spam = df[df['v1']=='spam']

df_ham = df[df['v1']=='ham']

print("Ham Dataset Shape:", df_ham.shape)

print("Spam Dataset Shape:", df_spam.shape)

Ham Dataset Shape: (4825, 5)
Spam Dataset Shape: (747, 5)


In [6]:
# downsampling ham dataset - take only random 747 example
# will use df_spam.shape[0] - 747
df_ham_downsampled = df_ham.sample(df_spam.shape[0])
df_ham_downsampled.shape

(747, 5)

In [7]:
# concating both dataset - df_spam and df_ham_balanced to create df_balanced dataset
df_balanced = pd.concat([df_spam , df_ham_downsampled])

In [8]:
df_balanced['v1'].value_counts()

spam    747
ham     747
Name: v1, dtype: int64

In [9]:
df_balanced.sample(10)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
3862,spam,Oh my god! I've found your number again! I'm s...,NaN,NaN,NaN
3858,spam,Win the newest åÒHarry Potter and the Order of...,NaN,NaN,NaN
3284,ham,They are just making it easy to pay back. I ha...,NaN,NaN,NaN
366,spam,"Update_Now - Xmas Offer! Latest Motorola, Sony...",NaN,NaN,NaN
5497,spam,"SMS SERVICES. for your inclusive text credits,...",NaN,NaN,NaN
5498,ham,Why did I wake up on my own &gt;:(,NaN,NaN,NaN
5155,ham,MY NEW YEARS EVE WAS OK. I WENT TO A PARTY WIT...,NaN,NaN,NaN
1520,spam,"URGENT! Your Mobile No was awarded a å£2,000 B...",NaN,NaN,NaN
909,spam,"January Male Sale! Hot Gay chat now cheaper, c...",NaN,NaN,NaN
3941,ham,She's borderline but yeah whatever.,NaN,NaN,NaN


## Preprocessing of Spam Detection Data

In [10]:
# creating numerical repersentation of category - one hot encoding
df_balanced['spam'] = df_balanced['v1'].apply(lambda x:1 if x=='spam' else 0)

In [11]:
# displaying data - spam -1 , ham-0
df_balanced.sample(4)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4,spam
11,spam,"SIX chances to win CASH! From 100 to 20,000 po...",NaN,NaN,NaN,1
1904,ham,Wah... Okie okie... Muz make use of e unlimite...,NaN,NaN,NaN,0
3787,spam,Want to funk up ur fone with a weekly new tone...,NaN,NaN,NaN,1
566,ham,Oooh bed ridden ey? What are YOU thinking of?,NaN,NaN,NaN,0


## Performing Train Test Split

In [12]:
# loading train test split
from sklearn.model_selection import train_test_split
X_train, X_test , y_train, y_test = train_test_split(df_balanced['v2'], df_balanced['spam'],
                                                    stratify = df_balanced['spam'])

## Model Creation

## Downloading Prerequisites

In [13]:
# downloading preprocessing files and model
bert_preprocessor = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
bert_encoder = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4')

## Creating Model

Having downloaded the bert model, we can now use Keras Functional API to build our model.

In [14]:
text_input = tf.keras.layers.Input(shape = (), dtype = tf.string, name = 'Inputs')
preprocessed_text = bert_preprocessor(text_input)
embeed = bert_encoder(preprocessed_text)
dropout = tf.keras.layers.Dropout(0.1, name = 'Dropout')(embeed['pooled_output'])
outputs = tf.keras.layers.Dense(1, activation = 'sigmoid', name = 'Dense')(dropout)
# creating final model
model = tf.keras.Model(inputs = [text_input], outputs = [outputs])

In [15]:
# check the summary of the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Inputs (InputLayer)            [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_type_ids':   0           ['Inputs[0][0]']                 
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                      

## Compiling And Training Model

In [16]:
Metrics = [tf.keras.metrics.BinaryAccuracy(name = 'accuracy'),
           tf.keras.metrics.Precision(name = 'precision'),
           tf.keras.metrics.Recall(name = 'recall')
           ]
# compiling our model
model.compile(optimizer ='adam',
               loss = 'binary_crossentropy',
               metrics = Metrics)

In [17]:
history = model.fit(X_train, y_train, epochs = 10)

Epoch 1/10
35/35 [==============================] - 148s 4s/step - loss: 0.6761 - accuracy: 0.5795 - precision: 0.5817 - recall: 0.5661
Epoch 2/10
35/35 [==============================] - 134s 4s/step - loss: 0.5342 - accuracy: 0.7973 - precision: 0.7780 - recall: 0.8321
Epoch 3/10
35/35 [==============================] - 132s 4s/step - loss: 0.4597 - accuracy: 0.8429 - precision: 0.8276 - recall: 0.8661
Epoch 4/10
35/35 [==============================] - 120s 3s/step - loss: 0.4166 - accuracy: 0.8607 - precision: 0.8447 - recall: 0.8839
Epoch 5/10
35/35 [==============================] - 120s 3s/step - loss: 0.3808 - accuracy: 0.8786 - precision: 0.8569 - recall: 0.9089
Epoch 6/10
35/35 [==============================] - 116s 3s/step - loss: 0.3513 - accuracy: 0.8866 - precision: 0.8805 - recall: 0.8946
Epoch 7/10
35/35 [==============================] - 116s 3s/step - loss: 0.3339 - accuracy: 0.8902 - precision: 0.8827 - recall: 0.9000
Epoch 8/10
35/35 [==============================

## Model Evaluation

In [18]:
# Evaluating performance
model.evaluate(X_test,y_test)

12/12 [==============================] - 42s 3s/step - loss: 0.2695 - accuracy: 0.9225 - precision: 0.9489 - recall: 0.8930


[0.26949289441108704,
 0.9224599003791809,
 0.9488636255264282,
 0.893048107624054]

## Model Prediction for Spam Detection with some random data

In [25]:
predict_text = ['We’d all like to get a $10,000 deposit on our bank accounts out of the blue, but winning a prize—especially if you’ve never entered a contest', 
                'Netflix is sending you a refund of $12.99. Please reply with your bank account and routing number to verify and get your refund', 
                'Your account is temporarily frozen. Please log in to to secure your account ',
                'The article was published on 18th August itself',
                'Although we are unable to give you an exact time-frame at the moment, I would request you to stay tuned for any updates.',
                'The image you sent is a UI bug, I can check that your article is marked as regular and is not in the monetization program.'
]

In [33]:
test_results = model.predict(predict_text)
output = np.where(test_results<0.5,'spam', 'ham')

1/1 [==============================] - 1s 889ms/step


In [34]:
output

array([['spam'],
       ['ham'],
       ['ham'],
       ['spam'],
       ['spam'],
       ['ham']], dtype='<U4')